# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f7f8c918af0>
├── 'a' --> tensor([[-1.0215, -0.2369, -0.2965],
│                   [-0.4776, -0.4529, -0.2229]])
└── 'x' --> <FastTreeValue 0x7f7f8c98c9a0>
    └── 'c' --> tensor([[ 1.1927, -0.9925, -0.0217,  1.4570],
                        [ 0.3784,  1.2944, -0.0908,  0.0272],
                        [-0.4535,  0.5005,  0.0339,  0.2225]])

In [4]:
t.a

tensor([[-1.0215, -0.2369, -0.2965],
        [-0.4776, -0.4529, -0.2229]])

In [5]:
%timeit t.a

74.1 ns ± 0.161 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f7f8c918af0>
├── 'a' --> tensor([[ 0.6278,  0.5862, -0.1813],
│                   [-1.1650,  0.6159, -1.5564]])
└── 'x' --> <FastTreeValue 0x7f7f8c98c9a0>
    └── 'c' --> tensor([[ 1.1927, -0.9925, -0.0217,  1.4570],
                        [ 0.3784,  1.2944, -0.0908,  0.0272],
                        [-0.4535,  0.5005,  0.0339,  0.2225]])

In [7]:
%timeit t.a = new_value

85.2 ns ± 0.371 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[-1.0215, -0.2369, -0.2965],
               [-0.4776, -0.4529, -0.2229]]),
    x: Batch(
           c: tensor([[ 1.1927, -0.9925, -0.0217,  1.4570],
                      [ 0.3784,  1.2944, -0.0908,  0.0272],
                      [-0.4535,  0.5005,  0.0339,  0.2225]]),
       ),
)

In [10]:
b.a

tensor([[-1.0215, -0.2369, -0.2965],
        [-0.4776, -0.4529, -0.2229]])

In [11]:
%timeit b.a

69.5 ns ± 0.187 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[ 0.3104,  0.3326, -0.8663],
               [ 1.0819, -0.4738,  0.2329]]),
    x: Batch(
           c: tensor([[ 1.1927, -0.9925, -0.0217,  1.4570],
                      [ 0.3784,  1.2944, -0.0908,  0.0272],
                      [-0.4535,  0.5005,  0.0339,  0.2225]]),
       ),
)

In [13]:
%timeit b.a = new_value

601 ns ± 2.57 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

1.01 µs ± 4.64 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

11.8 µs ± 87.1 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

167 µs ± 690 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

166 µs ± 981 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f7ee2cd4c40>
├── 'a' --> tensor([[[-1.0215, -0.2369, -0.2965],
│                    [-0.4776, -0.4529, -0.2229]],
│           
│                   [[-1.0215, -0.2369, -0.2965],
│                    [-0.4776, -0.4529, -0.2229]],
│           
│                   [[-1.0215, -0.2369, -0.2965],
│                    [-0.4776, -0.4529, -0.2229]],
│           
│                   [[-1.0215, -0.2369, -0.2965],
│                    [-0.4776, -0.4529, -0.2229]],
│           
│                   [[-1.0215, -0.2369, -0.2965],
│                    [-0.4776, -0.4529, -0.2229]],
│           
│                   [[-1.0215, -0.2369, -0.2965],
│                    [-0.4776, -0.4529, -0.2229]],
│           
│                   [[-1.0215, -0.2369, -0.2965],
│                    [-0.4776, -0.4529, -0.2229]],
│           
│                   [[-1.0215, -0.2369, -0.2965],
│                    [-0.4776, -0.4529, -0.2229]]])
└── 'x' --> <FastTreeValue 0x7f7ee92ad850>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

36.9 µs ± 285 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f7ee2d26070>
├── 'a' --> tensor([[-1.0215, -0.2369, -0.2965],
│                   [-0.4776, -0.4529, -0.2229],
│                   [-1.0215, -0.2369, -0.2965],
│                   [-0.4776, -0.4529, -0.2229],
│                   [-1.0215, -0.2369, -0.2965],
│                   [-0.4776, -0.4529, -0.2229],
│                   [-1.0215, -0.2369, -0.2965],
│                   [-0.4776, -0.4529, -0.2229],
│                   [-1.0215, -0.2369, -0.2965],
│                   [-0.4776, -0.4529, -0.2229],
│                   [-1.0215, -0.2369, -0.2965],
│                   [-0.4776, -0.4529, -0.2229],
│                   [-1.0215, -0.2369, -0.2965],
│                   [-0.4776, -0.4529, -0.2229],
│                   [-1.0215, -0.2369, -0.2965],
│                   [-0.4776, -0.4529, -0.2229]])
└── 'x' --> <FastTreeValue 0x7f7ee26a43a0>
    └── 'c' --> tensor([[ 1.1927, -0.9925, -0.0217,  1.4570],
                        [ 0.3784,  1.2944, -0.0908,  0.0272],
                 

In [23]:
%timeit t_cat(trees)

36 µs ± 1.04 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

64.6 µs ± 306 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    a: tensor([[[-1.0215, -0.2369, -0.2965],
                [-0.4776, -0.4529, -0.2229]],
       
               [[-1.0215, -0.2369, -0.2965],
                [-0.4776, -0.4529, -0.2229]],
       
               [[-1.0215, -0.2369, -0.2965],
                [-0.4776, -0.4529, -0.2229]],
       
               [[-1.0215, -0.2369, -0.2965],
                [-0.4776, -0.4529, -0.2229]],
       
               [[-1.0215, -0.2369, -0.2965],
                [-0.4776, -0.4529, -0.2229]],
       
               [[-1.0215, -0.2369, -0.2965],
                [-0.4776, -0.4529, -0.2229]],
       
               [[-1.0215, -0.2369, -0.2965],
                [-0.4776, -0.4529, -0.2229]],
       
               [[-1.0215, -0.2369, -0.2965],
                [-0.4776, -0.4529, -0.2229]]]),
    x: Batch(
           c: tensor([[[ 1.1927, -0.9925, -0.0217,  1.4570],
                       [ 0.3784,  1.2944, -0.0908,  0.0272],
                       [-0.4535,  0.5005,  0.0339,  0.2225]],
         

In [26]:
%timeit Batch.stack(batches)

90.8 µs ± 597 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    a: tensor([[-1.0215, -0.2369, -0.2965],
               [-0.4776, -0.4529, -0.2229],
               [-1.0215, -0.2369, -0.2965],
               [-0.4776, -0.4529, -0.2229],
               [-1.0215, -0.2369, -0.2965],
               [-0.4776, -0.4529, -0.2229],
               [-1.0215, -0.2369, -0.2965],
               [-0.4776, -0.4529, -0.2229],
               [-1.0215, -0.2369, -0.2965],
               [-0.4776, -0.4529, -0.2229],
               [-1.0215, -0.2369, -0.2965],
               [-0.4776, -0.4529, -0.2229],
               [-1.0215, -0.2369, -0.2965],
               [-0.4776, -0.4529, -0.2229],
               [-1.0215, -0.2369, -0.2965],
               [-0.4776, -0.4529, -0.2229]]),
    x: Batch(
           c: tensor([[ 1.1927, -0.9925, -0.0217,  1.4570],
                      [ 0.3784,  1.2944, -0.0908,  0.0272],
                      [-0.4535,  0.5005,  0.0339,  0.2225],
                      [ 1.1927, -0.9925, -0.0217,  1.4570],
                      [ 0.3784,  

In [28]:
%timeit Batch.cat(batches)

166 µs ± 817 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

377 µs ± 7.46 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
